In [87]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score,\
f1_score, confusion_matrix, ConfusionMatrixDisplay

from sklearn.ensemble import RandomForestClassifier


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [88]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [89]:
test = pd.read_csv('/kaggle/input/titanic/test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [90]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [91]:
display(train.shape)
display(test.shape)

(891, 12)

(418, 11)

In [92]:
display(train.dtypes)

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [93]:
display(train['Cabin'].unique())
display(train['SibSp'].unique())
display(train[train['Survived']==1].groupby('SibSp')[['Survived']].apply(lambda a: a.value_counts()))
display(train[train['Survived']==0].groupby('SibSp')[['Survived']].apply(lambda a: a.value_counts()))

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62 C64',

array([1, 0, 3, 4, 2, 5, 8])

Survived,1
SibSp,
0,210
1,112
2,13
3,4
4,3


Survived,0
SibSp,
0,398
1,97
2,15
3,12
4,15
5,5
8,7


In [94]:
bins = [0, 100, 200, 300, 400, 500, 600]
labels = ['0-100', '100-200', '200-300', '300-400', '400-500', '500-600']
train['Fare Group'] = pd.cut(train['Fare'], bins=bins, labels=labels, right=False)
test['Fare Group'] = pd.cut(test['Fare'], bins=bins, labels=labels, right=False)
display(train[train['Survived']==1].groupby(['Fare Group'],observed=True).agg({'Survived': 'value_counts'}).to_dict())
display(train[train['Survived']==0].groupby(['Fare Group'],observed=False)[['Survived']].apply(lambda a: a.value_counts()))

{'Survived': {('0-100', 1): 303,
  ('100-200', 1): 25,
  ('200-300', 1): 11,
  ('500-600', 1): 3}}

Fare Group  Survived
0-100       0           535
100-200     0             8
200-300     0             6
Name: count, dtype: int64

In [104]:
def get_survived_probabilities(survived, features:[]):
    f_grpby = train[train['Survived']==survived].groupby(features,observed=True).agg({'Survived': 'value_counts'}).to_dict()
    c_dict={}
    total = 0
    for k,v in f_grpby['Survived'].items():
        c_dict[k[0]] = v
        total += v
    p_dict = {key: round((value / total), 4) for key, value in c_dict.items()}
    return p_dict

In [105]:
train.loc[train['Survived'] == 1, "Fare_Survived_Probs"] = train["Fare Group"].map(get_survived_probabilities(1,['Fare Group']))
train.loc[train['Survived'] == 0, "Fare_Survived_Probs"] = train["Fare Group"].map(get_survived_probabilities(0,['Fare Group']))
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare Group,Fare_Survived_Probs
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0-100,0.9745
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0-100,0.8860
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0-100,0.8860
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0-100,0.8860
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0-100,0.9745


In [106]:
train["Fare_Survived_Probs"].values

array([0.9745, 0.886 , 0.886 , 0.886 , 0.9745, 0.9745, 0.9745, 0.9745,
       0.886 , 0.886 , 0.886 , 0.886 , 0.9745, 0.9745, 0.9745, 0.886 ,
       0.9745, 0.886 , 0.9745, 0.886 , 0.9745, 0.886 , 0.886 , 0.886 ,
       0.9745, 0.886 , 0.9745, 0.0109, 0.886 , 0.9745, 0.9745, 0.0731,
       0.886 , 0.9745, 0.9745, 0.9745, 0.886 , 0.9745, 0.9745, 0.886 ,
       0.9745, 0.9745, 0.9745, 0.886 , 0.886 , 0.9745, 0.9745, 0.886 ,
       0.9745, 0.9745, 0.9745, 0.9745, 0.886 , 0.886 , 0.9745, 0.886 ,
       0.886 , 0.9745, 0.886 , 0.9745, 0.9745, 0.886 , 0.9745, 0.9745,
       0.9745, 0.886 , 0.886 , 0.9745, 0.886 , 0.9745, 0.9745, 0.9745,
       0.9745, 0.9745, 0.886 , 0.9745, 0.9745, 0.9745, 0.886 , 0.886 ,
       0.9745, 0.886 , 0.886 , 0.9745, 0.886 , 0.886 , 0.9745, 0.9745,
       0.0322, 0.9745, 0.9745, 0.9745, 0.9745, 0.9745, 0.9745, 0.9745,
       0.9745, 0.886 , 0.886 , 0.9745, 0.9745, 0.9745, 0.9745, 0.9745,
       0.9745, 0.9745, 0.886 , 0.886 , 0.9745, 0.886 , 0.9745, 0.9745,
      

In [ ]:
display(train['Pclass'].unique())
display(train[train['Survived']==1].groupby('Pclass')[['Survived']].apply(lambda a: a.value_counts()))
display(train[train['Survived']==0].groupby('Pclass')[['Survived']].apply(lambda a: a.value_counts()))

In [ ]:
display(train[train['Survived']==1].groupby(['Sex', 'Age'])[['Survived']].apply(lambda a: a.value_counts()))
display(train[train['Survived']==0].groupby(['Sex', 'Age'])[['Survived']].apply(lambda a: a.value_counts()))

In [ ]:
total_females_survived = train[train['Survived']==1].groupby('Sex')[['Survived']].apply(lambda a: a.value_counts()).loc['female'].values[0]
total_males_survived = train[train['Survived']==1].groupby('Sex')[['Survived']].apply(lambda a: a.value_counts()).loc['male'].values[0]

In [ ]:
train[train['Sex']=='male' && train['Sex']].apply(lambda a: a[''])

In [ ]:
display(train['Age'].describe())
display(train['Age'].unique())
# Define age bins
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80]
labels = ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80']

In [ ]:
train['Age Group'] = pd.cut(train['Age'], bins=bins, labels=labels, right=False)
test['Age Group'] = pd.cut(test['Age'], bins=bins, labels=labels, right=False)
display(train[train['Survived']==1].groupby('Age Group')[['Survived']].apply(lambda a: a.value_counts()))
display(train[train['Survived']==0].groupby('Age Group')[['Survived']].apply(lambda a: a.value_counts()))
#display(test.head())

In [ ]:

display(train['Embarked'].unique())
embarked_mapping = {
    'nan': 0,
     np.nan: 0,
    'S': 1,
    'C': 2,
    'Q': 3
}
train_plne = train.copy()
test_plne = test.copy()
train_plne['Embarked'] = train_plne['Embarked'].replace(embarked_mapping)
test_plne['Embarked'] = test_plne['Embarked'].replace(embarked_mapping)
train_median_age = train_plne['Age'].median()
test_median_age = test_plne['Age'].median()

# Impute NaN values in the Age column with the median age
train_plne['Age'].fillna(train_median_age, inplace=True)
test_plne['Age'].fillna(test_median_age, inplace=True)
display(train_plne.head())
display(test_plne.head())

In [ ]:
test['Embarked'].unique()

In [ ]:
train_plne = pd.get_dummies(train_plne, columns=['Sex','Embarked','Pclass','Age Group', 'Fare Group','SibSp'], prefix=['Sex','Embarked','Pclass','Age Group', 'Fare Group','SibSp'], dtype=int)
test_plne = pd.get_dummies(test_plne, columns=['Sex','Embarked','Pclass', 'Age Group', 'Fare Group', 'SibSp'], prefix=['Sex','Embarked','Pclass','Age Group', 'Fare Group', 'SibSp'], dtype=int)

In [ ]:
train_plne.head()

In [ ]:
nan_counts = train_plne.isna().sum()
display(nan_counts)
nan_counts = test_plne.isna().sum()
display(nan_counts)

In [ ]:
train_plne.dtypes

Check if the age group is a feature that can be used, dummy variable
Cabin seems to have 40% values missing but maybe useful to show likelihood of survival, can be a dummy variable
Maybe PClass should be a dummy variable
We should also look at the fares and see if passengers with higher or lower fare had more probability of survival maybe can be dummy variable based on range.

In [ ]:
train_plne.columns

In [ ]:
drop_columns_train = ['Name','Age','Parch','Fare','Ticket','Cabin','Embarked_0']
drop_columns_test = ['Name','Age','Parch','Fare','Ticket','Cabin']
train_plne.columns

In [ ]:
import pandas as pd
train_num = train_plne.drop(drop_columns_train, axis=1)
test_num = test_plne.drop(drop_columns_test, axis=1)
X = train_num.drop(['Survived','PassengerId'], axis=1)
y = train_num['Survived'].to_frame()

In [ ]:
print(X.columns)
print(test_num.columns)

In [ ]:
import torch
xs = torch.Tensor(X.values)
ys = torch.Tensor(y.values)
xs[0]

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
# Create a TensorDataset and DataLoader
X_train, X_val, y_train, y_val = train_test_split(xs, ys, test_size=0.2, random_state=42, shuffle=True)

# Create TensorDatasets and DataLoaders
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = TensorDataset(X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False) #no need to shuffle validation.

In [ ]:
len(xs[0])

In [ ]:
import torch.nn as nn

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # First fully connected layer
        self.fc2 = nn.Linear(hidden_size, hidden_size) # Second fully connected layer
        self.fc3 = nn.Linear(hidden_size, output_size) # Third fully connected layer
        self.sigmoid = nn.Sigmoid() #Sigmoid activation for binary classification

    def forward(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        return out

# Instantiate the model
input_size = len(xs[0])
hidden_size = len(xs[0]) * 2
output_size = 1  # Binary classification, so one output neuron
model = SimpleNN(input_size, hidden_size, output_size)

In [ ]:
import torch.optim as optim
criterion = nn.BCELoss()  # Binary cross-entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

In [ ]:
epochs = 1000

for epoch in range(epochs):
    # Training
    model.train()  # Set the model to training mode
    for inputs, labels in train_dataloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs).squeeze(0) #squeeze to remove extra dimension.

        # Calculate the loss
        loss = criterion(outputs, labels.squeeze(0)) #convert labels to float

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient calculations during validation
        for inputs, labels in val_dataloader:
            outputs = model(inputs).squeeze(0)
            val_loss += criterion(outputs, labels.squeeze(0)).item()
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels.squeeze(0)).sum().item()

    avg_val_loss = val_loss / len(val_dataloader)
    accuracy = 100 * correct / total

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {loss.item():.4f}, Val Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.2f}%')

print('Training finished!')

In [ ]:
test_num.drop('PassengerId', axis=1)

In [ ]:
X.values.ravel()

In [ ]:
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X.values, y.values)
predictions = model.predict(test_num.drop('PassengerId', axis=1).values)
output = pd.DataFrame({'PassengerId': test_num.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

In [ ]:
output

In [ ]:
model.eval()  # Set to evaluation mode

In [ ]:
input_features = test_num.drop('PassengerId', axis=1).iloc[5].values
len(input_features)

In [ ]:
import torch

# Example input data
new_input = torch.Tensor(input_features) 

# Make predictions
with torch.no_grad():
    predictions = model(new_input)

# Process the predictions
predicted_label = (predictions > 0.5).float()  # Binary classification example
print("Predictions:", predictions)
print("Predicted label:", predicted_label)

In [ ]:
test_dataloader = TensorDataset(torch.Tensor(test_num.drop('PassengerId', axis=1).values))

In [ ]:
ids = []
predictions = []
with torch.no_grad():  # Disable gradient calculations
    for (inputs,id) in zip(test_dataloader, test['PassengerId'].values):
        inputs = inputs[0] #dataloader returns a tuple, so we need to grab the first element.
        outputs = model(inputs).squeeze(0)  # Get model outputs
        predicted = (outputs > 0.5).float()  # Apply threshold for binary classification
        predictions.append(predicted.item())  # Collect predictions

# --- Processing predictions ---
print("Predictions:", predictions)

In [ ]:
ids = []
predictions = []
for (X,id) in zip(test_dataloader, test['PassengerId'].values):
    label = model(X[0])
    ids.append(id)
    predictions.append((int((label > 0.5).float().sum().item())))

In [ ]:
# Create a DataFrame from the lists
df = pd.DataFrame({
    'PassengerId': ids,
    'Survived': predictions
})

# Save the DataFrame to a CSV file without the default index
df.to_csv('submission.csv', index=False)